## Chapter 13-15

In [1]:
!pip install pdfplumber
!pip install pyspellchecker
!pip install spacy
import pdfplumber


In [2]:
## Path to the pdf
pdf_path = "E:\SCHOOL\Phase 5\Kenya Constitution\Kenya Constitution.pdf"

with pdfplumber.open(pdf_path) as pdf:
  for page in pdf.pages:
      text = page.extract_text()

In [3]:
## Function to extract text from specific pages

def extract_specific_pages(pdf_path, start_page, end_page):
  with pdfplumber.open(pdf_path) as pdf:
    num_pages = len(pdf.pages) # Get the total number of pages in the PDF.
    extracted_text = ""
    # Adjust page numbers to be within the valid range
    start_page = max(0, min(start_page, num_pages - 1))
    end_page = max(0, min(end_page, num_pages))
    for page_num in range(start_page, end_page):
      page = pdf.pages[page_num]
      extracted_text += page.extract_text() + "\n"
    return extracted_text

### Chapter 13

In [4]:
chapter_13 = extract_specific_pages(pdf_path, 138,143)
chapter_13

'Constitution of Kenya, 2010 139\nCHAPTER THIRTEEN—THE PUBLIC SERVICE\nPART 1—VALUES AND PRINCIPLES OF PUBLIC SERVICE\nValues and principles of public service.\n232. (1) The values and principles of public service include—\n(a) high standards of professional ethics;\n(b) efficient, effective and economic use of resources;\n(c) responsive, prompt, effective, impartial and equitable\nprovision of services;\n(d) involvement of the people in the process of policy making;\n(e) accountability for administrative acts;\n(f) transparency and provision to the public of timely, accurate\ninformation;\n(g) subject to paragraphs (h) and (i), fair competition and merit as\nthe basis of appointments and promotions;\n(h) representation of Kenya’s diverse communities; and\n(i) affording adequate and equal opportunities for appointment,\ntraining and advancement, at all levels of the public service,\nof—\n(i) men and women;\n(ii) the members of all ethnic groups; and\n(iii) persons with disabilities.\n(

Since chapter 14 is included in the last page of cahpter 13, let's trim it to only contain information on chapter 13

In [5]:
chapter_13_trimmed = chapter_13.split("CHAPTER FOURTEEN")[0].strip()
chapter_13_trimmed

'Constitution of Kenya, 2010 139\nCHAPTER THIRTEEN—THE PUBLIC SERVICE\nPART 1—VALUES AND PRINCIPLES OF PUBLIC SERVICE\nValues and principles of public service.\n232. (1) The values and principles of public service include—\n(a) high standards of professional ethics;\n(b) efficient, effective and economic use of resources;\n(c) responsive, prompt, effective, impartial and equitable\nprovision of services;\n(d) involvement of the people in the process of policy making;\n(e) accountability for administrative acts;\n(f) transparency and provision to the public of timely, accurate\ninformation;\n(g) subject to paragraphs (h) and (i), fair competition and merit as\nthe basis of appointments and promotions;\n(h) representation of Kenya’s diverse communities; and\n(i) affording adequate and equal opportunities for appointment,\ntraining and advancement, at all levels of the public service,\nof—\n(i) men and women;\n(ii) the members of all ethnic groups; and\n(iii) persons with disabilities.\n(

In [6]:
import spacy

In [7]:
## load spacy
nlp = spacy.load("en_core_web_sm")

## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What are Values and principles of public service?"
processed_query= preprocess_query(user_query)
print(processed_query)


value principle public service


 Now let's split the chapter into three different sections which will increase the chatbot's accuracy in answering questions.

In [45]:
def split_chapter(chapter_text):
  #split at key headings and split extra white spaces

  sections = {
      "principle":[],
      "public service commission":[],
      "functions":[],
      "staffing":[],
      "protection":[],
      "teacher service commission":[]
  }

  ## Split by new lines to process line by line
  lines = chapter_text.splitlines()
  current_section = None

  for line in lines:
    line = line.strip()
    # Use line instead of stripped_line
    if line.startswith("Values and principles of public service"):
      current_section = "principle"
    elif line.startswith("The Public Service Commission"):
        current_section = "public service commission"
    elif line.startswith("Functions and powers of the Public Service Commission"):
        current_section = "functions"
    elif line.startswith("Staffing of county governments"):
        current_section = "staffing"
    elif line.startswith("Protection of public officers"):
        current_section = "protection"
    elif line.startswith("Teachers Service Commission"):
            current_section = "teacher service commission"

    # Append line to the current section if it's set
    if current_section:
        sections[current_section].append(line) # Use line instead of stripped_line
  ## Join each section into single string
  for key in sections:
    sections[key] = "\n".join(sections[key])
  return sections

chapter_13_sections = split_chapter(chapter_13_trimmed)

Question and Answer mapping based on key phrases and corresponding sections.

In [46]:
## Define the sections variable with Chapter 13
sections = chapter_13_sections


#Define the Q&A mapping
qa_mapping = {
    "principle": "Values and principles of public service",
    "public service commission": "The Public Service Commission",
    "functions": "Functions and powers of the Public Service Commission",
    "staffing" : "Staffing of county governments",
    "protection": "Protection of public officers",
    "teacher service commission": "Teachers Service Commision"
}

##Update the Q&A system to use preprocessed queries
def answer_question_nlp(query):
  # Preprocess the user query
  processed_query = preprocess_query(query)
  # Debug line
  print(f"preprocessed_query : {processed_query}")

  #Search for a key in QA_mapping that matches the preprocessed query
  for key, value in qa_mapping.items(): # Iterate through key-value pairs of QA_mapping
    print(f"key : {key}")
    if key in processed_query:
      print(f"key : {key}")
    ## Return the relevant section in the text
      return sections[key] # Access sections using the full section name (value in QA_mapping)
  return "Sorry, I couldn't find an answer to your question."

##Example
user_query = "What about teacher service commission?"
answer = answer_question_nlp(user_query) # Pass the user_query
print(answer)


preprocessed_query : teacher service commission
key : principle
key : public service commission
key : functions
key : staffing
key : protection
key : teacher service commission
key : teacher service commission
Teachers Service Commission.
237. (1) There is established the Teachers Service Commission.
(2) The functions of the Commission are—
(a) to register trained teachers;
(b) to recruit and employ registered teachers;
Constitution of Kenya, 2010 143
(c) to assign teachers employed by the Commission for service in
any public school or institution;
(d) to promote and transfer teachers;
(e) to exercise disciplinary control over teachers; and
(f) to terminate the employment of teachers.
(3) The Commission shall—
(a) review the standards of education and training of persons
entering the teaching service;
(b) review the demand for and the supply of teachers; and
(c) advise the national government on matters relating to the
teaching profession.


### Chapter 14


In [12]:
chapter_14 = extract_specific_pages(pdf_path, 142,150)
chapter_14

'Constitution of Kenya, 2010 143\n(c) to assign teachers employed by the Commission for service in\nany public school or institution;\n(d) to promote and transfer teachers;\n(e) to exercise disciplinary control over teachers; and\n(f) to terminate the employment of teachers.\n(3) The Commission shall—\n(a) review the standards of education and training of persons\nentering the teaching service;\n(b) review the demand for and the supply of teachers; and\n(c) advise the national government on matters relating to the\nteaching profession.\nCHAPTER FOURTEEN—NATIONAL SECURITY\nPART 1—NATIONAL SECURITY ORGANS\nPrinciples of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following princi

Since Chapter 14 starts in the same page as the end of chapter 13.Let's trim it.


In [13]:
# Trim the chapter_14 text to start from "CHAPTER FOURTEEN"
chapter_14_trimmed = chapter_14[chapter_14.find("CHAPTER FOURTEEN"):].strip()

## Further trim to remove Chapter 15 if it's present on the same page
if "CHAPTER FIFTEEN" in chapter_14_trimmed:
  chapter_14_trimmed = chapter_14_trimmed.split("CHAPTER FIFTEEN")[0].strip()


chapter_14_trimmed

'CHAPTER FOURTEEN—NATIONAL SECURITY\nPART 1—NATIONAL SECURITY ORGANS\nPrinciples of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following principles—\n(a) national security is subject to the authority of this Constitution\nand Parliament;\n(b) national security shall be pursued in compliance with the law\nand with the utmost respect for the rule of law, democracy,\nhuman rights and fundamental freedoms;\n(c) in performing their functions and exercising their powers,\nnational security organs shall respect the diverse culture of\nthe communities within Kenya; and\n(d) recruitment by the national security organs shall reflect the\ndiversity of the Kenyan people in equitable propo

In [14]:
## load spacy
nlp = spacy.load("en_core_web_sm")

## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What are the principles of national security?"
preprocess_query= preprocess_query(user_query)
print(preprocess_query)


principle national security


Let's split the chapter into sections.


In [15]:
chapter_14_trimmed

'CHAPTER FOURTEEN—NATIONAL SECURITY\nPART 1—NATIONAL SECURITY ORGANS\nPrinciples of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following principles—\n(a) national security is subject to the authority of this Constitution\nand Parliament;\n(b) national security shall be pursued in compliance with the law\nand with the utmost respect for the rule of law, democracy,\nhuman rights and fundamental freedoms;\n(c) in performing their functions and exercising their powers,\nnational security organs shall respect the diverse culture of\nthe communities within Kenya; and\n(d) recruitment by the national security organs shall reflect the\ndiversity of the Kenyan people in equitable propo

In [16]:
def split_chapter(chapter_text):
    # Define sections with lowercase keys
    sections = {
        "national security": [],
        "national security organs": [],
        "national security council": [],
        "defence forces": [],
        "national intelligence service": [],
        "national police service": [],
        "functions national police service": [],
        "command national police service": [],
        "national police service commission": [],
        "other police services": []
    }

    # Split by new lines to process line by line
    lines = chapter_text.splitlines()


    current_section = None

    for line in lines:
        stripped_line = line.strip()

        # Match line starts with capitalized section headers
        if stripped_line.startswith("Principles of national security"):
            current_section = "national security"
        elif stripped_line.startswith("National security organs"):
            current_section = "national security organs"
        elif stripped_line.startswith("Establishment of the National Security Council"):
            current_section = "national security council"
        elif stripped_line.startswith("Establishment of Defence Forces and Defence Council"):
            current_section = "defence forces"
        elif stripped_line.startswith("Establishment of National Intelligence Service"):
            current_section = "national intelligence service"
        elif stripped_line.startswith("Establishment of the National Police Service"):
            current_section = "national police service"
        elif stripped_line.startswith("Objects and Functions of the National Police Service"):
            current_section = "functions national police service"
        elif stripped_line.startswith("Command of the National Police Service"):
            current_section = "command national police service"
        elif stripped_line.startswith("National Police Service Commission"):
            current_section = "national police service commission"
        elif stripped_line.startswith("Other Police Services"):
            current_section = "other police services"

        # Append line to the current section if it's set
        if current_section:
            sections[current_section].append(line)

    # Join each section into a single string
    for key in sections:
        sections[key] = "\n".join(sections[key])

    return sections

# Apply the function to the chapter 14
chapter_14_sections = split_chapter(chapter_14_trimmed)


In [17]:
chapter_14_sections

{'national security': 'Principles of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following principles—\n(a) national security is subject to the authority of this Constitution\nand Parliament;\n(b) national security shall be pursued in compliance with the law\nand with the utmost respect for the rule of law, democracy,\nhuman rights and fundamental freedoms;\n(c) in performing their functions and exercising their powers,\nnational security organs shall respect the diverse culture of\nthe communities within Kenya; and\n(d) recruitment by the national security organs shall reflect the\ndiversity of the Kenyan people in equitable proportions.',
 'national security organs': 'Nationa

In [18]:
print("Principles of National Security Section:\n", chapter_14_sections["national security"])
print("National security organs Section:\n", chapter_14_sections["national security organs"])

Principles of National Security Section:
 Principles of national security.
238. (1) National security is the protection against internal and
external threats to Kenya’s territorial integrity and sovereignty, its
people, their rights, freedoms, property, peace, stability and prosperity,
and other national interests.
(2) The national security of Kenya shall be promoted and
guaranteed in accordance with the following principles—
(a) national security is subject to the authority of this Constitution
and Parliament;
(b) national security shall be pursued in compliance with the law
and with the utmost respect for the rule of law, democracy,
human rights and fundamental freedoms;
(c) in performing their functions and exercising their powers,
national security organs shall respect the diverse culture of
the communities within Kenya; and
(d) recruitment by the national security organs shall reflect the
diversity of the Kenyan people in equitable proportions.
National security organs Section:
 N

Question and Answer mapping for Chapter 14


In [19]:
chapter_14_sections

{'national security': 'Principles of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following principles—\n(a) national security is subject to the authority of this Constitution\nand Parliament;\n(b) national security shall be pursued in compliance with the law\nand with the utmost respect for the rule of law, democracy,\nhuman rights and fundamental freedoms;\n(c) in performing their functions and exercising their powers,\nnational security organs shall respect the diverse culture of\nthe communities within Kenya; and\n(d) recruitment by the national security organs shall reflect the\ndiversity of the Kenyan people in equitable proportions.',
 'national security organs': 'Nationa

In [20]:
qa_mapping = {
    "national security": "national security",
    "national security organs": "national security organs",
    "national security council": "national security council",
    "defence forces and defence council": "defence forces",
    "national intelligence service": "national intelligence service",
    "national police service": "national police service",
    "functions national police service": "functions national police service",
    "command national police service": "command national police service",
    "national police service commission": "national police service commission",
    "other police services": "other police services"
}

# Preprocess the user query using spaCy
def preprocess_query(query):
    # Parse the query with spaCy
    doc = nlp(query)
    # Normalize the query: lowercase, lemmatize, and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)

    # Debug
    print(f"Processed query: {processed_query}")

    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Debug line
            print(f"key: {key}")
            # Return the relevant section in the text
            return sections[key]

    return "Sorry, I couldn't find an answer to your question."

##Example
user_query = "What are the principles of national security?"
answer = answer_question_nlp(user_query,chapter_14_sections, qa_mapping) # Pass the user_query
answer

Processed query: principle national security
key: national security


'Principles of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following principles—\n(a) national security is subject to the authority of this Constitution\nand Parliament;\n(b) national security shall be pursued in compliance with the law\nand with the utmost respect for the rule of law, democracy,\nhuman rights and fundamental freedoms;\n(c) in performing their functions and exercising their powers,\nnational security organs shall respect the diverse culture of\nthe communities within Kenya; and\n(d) recruitment by the national security organs shall reflect the\ndiversity of the Kenyan people in equitable proportions.'

### Chapter 14 done.



### Chapter 15

In [21]:
chapter_15 = extract_specific_pages(pdf_path, 148,155)
chapter_15

'Constitution of Kenya, 2010 149\nCommission.\n(2) The Commission consists of—\n(a) the following persons, each appointed by the President—\n(i) a person who is qualified to be appointed as a High\nCourt Judge;\n(ii) two retired senior police officers; and\n(iii) three persons of integrity who have served the public with\ndistinction;\n(b) the Inspector-General of the National Police Service; and\n(c) both Deputy Inspectors-General of the National Police\nService.\n(3) The Commission shall—\n(a) recruit and appoint persons to hold or act in offices in the\nservice, confirm appointments, and determine promotions\nand transfers within the National Police Service;\n(b) observing due process, exercise disciplinary control over and\nremove persons holding or acting in offices within the\nService; and\n(c) perform any other functions prescribed by national legislation.\n(4) The composition of the National Police Service shall reflect\nthe regional and ethnic diversity of the people of Kenya.

In [22]:
# Trim the chapter_15 text to start from "CHAPTER FIFTEEN"
chapter_15_trimmed = chapter_15[chapter_15.find("CHAPTER FIFTEEN"):].strip()

# Further trim to remove Chapter 16 if it's present on the same page
if "CHAPTER SIXTEEN" in chapter_15_trimmed:
  chapter_15_trimmed = chapter_15_trimmed.split("CHAPTER SIXTEEN")[0].strip()

chapter_15_trimmed

'CHAPTER FIFTEEN—COMMISSIONS AND INDEPENDENT\nOFFICES\nApplication of Chapter.\n248. (1) This Chapter applies to the commissions specified in\nclause (2) and the independent offices specified in clause (3), except\nto the extent that this Constitution provides otherwise.\n(2) The commissions are—\n150 Constitution of Kenya, 2010\n(a) the Kenya National Human Rights and Equality Commission;\n(b) the National Land Commission;\n(c) the Independent Electoral and Boundaries Commission;\n(d) the Parliamentary Service Commission;\n(e) the Judicial Service Commission;\n(f) the Commission on Revenue Allocation;\n(g) the Public Service Commission;\n(h) the Salaries and Remuneration Commission;\n(i) the Teachers Service Commission; and\n(j) the National Police Service Commission.\n(3) The independent offices are—\n(a) the Auditor-General; and\n(b) the Controller of Budget.\nObjects, authority and funding of commissions and independent\noffices.\n249. (1) The objects of the commissions and the ind

In [23]:
##Further trim to remove Chapter 16 if it's present on the same page

if "CHAPTER SIXTEEN" in chapter_15_trimmed:
  chapter_15_trimmed = chapter_15_trimmed.split("CHAPTER SIXTEEN")[0].strip()

chapter_15_trimmed

'CHAPTER FIFTEEN—COMMISSIONS AND INDEPENDENT\nOFFICES\nApplication of Chapter.\n248. (1) This Chapter applies to the commissions specified in\nclause (2) and the independent offices specified in clause (3), except\nto the extent that this Constitution provides otherwise.\n(2) The commissions are—\n150 Constitution of Kenya, 2010\n(a) the Kenya National Human Rights and Equality Commission;\n(b) the National Land Commission;\n(c) the Independent Electoral and Boundaries Commission;\n(d) the Parliamentary Service Commission;\n(e) the Judicial Service Commission;\n(f) the Commission on Revenue Allocation;\n(g) the Public Service Commission;\n(h) the Salaries and Remuneration Commission;\n(i) the Teachers Service Commission; and\n(j) the National Police Service Commission.\n(3) The independent offices are—\n(a) the Auditor-General; and\n(b) the Controller of Budget.\nObjects, authority and funding of commissions and independent\noffices.\n249. (1) The objects of the commissions and the ind

In [24]:
## preprocess the user query using spacy

def preprocess_query(query):
  ## Parse the query with spacy
  doc = nlp(query)
  ## Normalize the query: lowercase,lemmatize and remove stopwords
  tokens = [token.lemma_.lower()for token in doc if not token.is_stop and not token.is_punct]
  return " ".join(tokens)

## Example
user_query = "What are the commissions and independent offices?"
preprocess_query= preprocess_query(user_query)
print(preprocess_query)

commission independent office


Now let's split chapter fifteen into sections.

In [25]:
def split_chapter_15(chapter_text):
  #split at key headings and split extra white spaces
  # Initialize sections as a dictionary with list values
  sections = {
        "application of chapter": [],
        "commission independent office": [],
        "term of office": [],
        "removal from office": [],
        "general function power": [],
        "incorporation commission independent office": [],
        "reporting commission independent office": []
    }

  # Split by new lines to process line by line
  lines = chapter_text.splitlines()
  current_section = None

  for line in lines:
    stripped_line = line.strip()

    if line.startswith("Application of Chapter"):
          current_section = "application of chapter"
    elif line.startswith("Objects,authority and funding of commissions and independent offices"):
          current_section = "commission independent office"
    elif line.startswith("Composition, appointment and terms of office"):
          current_section = "term of office"
    elif line.startswith("Removal from office"):
          current_section = "removal from office"
    elif line.startswith("General functions and powers"):
          current_section = "general function power"
    elif line.startswith("Incorporation of commissions and independent offices"):
          current_section = "incorporation commission independent office"
    elif line.startswith("Reporting by commissions and independent offices"):
          current_section = "reporting commission independent office"
          ## Append line to the current section if it's set

    if current_section:
            sections[current_section].append(stripped_line)

  # Join each section into single string
  for key in sections:
        sections[key] = "\n".join(sections[key])
  return sections

chapter_15_sections = split_chapter_15(chapter_15_trimmed)
 ## Example
print("Application of Chapter Section:\n", chapter_15_sections["application of chapter"])
print("Objects,authority and funding of commissions and independent offices Section:\n", chapter_15_sections["commission independent office"])

Application of Chapter Section:
 Application of Chapter.
248. (1) This Chapter applies to the commissions specified in
clause (2) and the independent offices specified in clause (3), except
to the extent that this Constitution provides otherwise.
(2) The commissions are—
150 Constitution of Kenya, 2010
(a) the Kenya National Human Rights and Equality Commission;
(b) the National Land Commission;
(c) the Independent Electoral and Boundaries Commission;
(d) the Parliamentary Service Commission;
(e) the Judicial Service Commission;
(f) the Commission on Revenue Allocation;
(g) the Public Service Commission;
(h) the Salaries and Remuneration Commission;
(i) the Teachers Service Commission; and
(j) the National Police Service Commission.
(3) The independent offices are—
(a) the Auditor-General; and
(b) the Controller of Budget.
Objects, authority and funding of commissions and independent
offices.
249. (1) The objects of the commissions and the independent
offices are to—
(a) protect the so

In [26]:
chapter_15_sections.keys()

dict_keys(['application of chapter', 'commission independent office', 'term of office', 'removal from office', 'general function power', 'incorporation commission independent office', 'reporting commission independent office'])

Question and Answer mapping based on key phrases and corresponding section in chapter 15

In [27]:
chapter_15_sections

{'application of chapter': 'Application of Chapter.\n248. (1) This Chapter applies to the commissions specified in\nclause (2) and the independent offices specified in clause (3), except\nto the extent that this Constitution provides otherwise.\n(2) The commissions are—\n150 Constitution of Kenya, 2010\n(a) the Kenya National Human Rights and Equality Commission;\n(b) the National Land Commission;\n(c) the Independent Electoral and Boundaries Commission;\n(d) the Parliamentary Service Commission;\n(e) the Judicial Service Commission;\n(f) the Commission on Revenue Allocation;\n(g) the Public Service Commission;\n(h) the Salaries and Remuneration Commission;\n(i) the Teachers Service Commission; and\n(j) the National Police Service Commission.\n(3) The independent offices are—\n(a) the Auditor-General; and\n(b) the Controller of Budget.\nObjects, authority and funding of commissions and independent\noffices.\n249. (1) The objects of the commissions and the independent\noffices are to—\n

In [50]:
chapter_15_sections['term of office']

'Composition, appointment and terms of office.\n250. (1) Each commission shall consist of at least three, but not\nmore than nine, members.\n(2) The chairperson and each member of a commission, and the\nholder of an independent office, shall be—\n(a) identified and recommended for appointment in a manner\nprescribed by national legislation;\n(b) approved by the National Assembly; and\n(c) appointed by the President.\n(3) To be appointed, a person shall have the specific\nqualifications required by this Constitution or national legislation.\n(4) Appointments to commissions and independent offices shall\ntake into account the national values referred to in Article 10, and the\nprinciple that the composition of the commissions and offices, taken as\na whole, shall reflect the regional and ethnic diversity of the people of\nKenya.\n(5) A member of a commission may serve on a part-time basis.\n(6) A member of a commission, or the holder of an independent\noffice—\n(a) unless ex officio, sha

In [48]:
# Define sections with lemmatized keys
sections = chapter_15_sections

# Ensure qa_mapping uses lemmatized keys
qa_mapping = {
    "application of chapter": "application of chapter",
    "commission independent office": "commission independent office",  # Use the actual key from 'sections'
    "term of office": "term of office",
    "removal from office": "removal from office",
    "general function power": "general function power",
    "incorporation commission independent office": "incorporation commission independent office",
    "reporting commissions independent office": "reporting commission independent office"
}


def preprocess_query(query):
    ## Parse the query with spacy
    doc = nlp(query)
    ## Normalize the query: lowercase,lemmatize and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)


# Update the Q&A system to use preprocessed queries
def answer_question_nlp(query, sections, qa_mapping):
    # Preprocess the user query
    processed_query = preprocess_query(query)

    # Debug
    print(f"Processed query: {processed_query}")

    # Search for a key in qa_mapping that matches the preprocessed query
    for key in qa_mapping:
        if key in processed_query:
            # Debug line
            print(f"key: {key}")
            # Return the relevant section in the text
            return sections[key]  # Access using the original key
    return "Sorry, I couldn't find an answer to your question."

# Example usage
user_query = "What about the commission independent office?"
answer = answer_question_nlp(user_query, sections, qa_mapping)
print(answer)

Processed query: commission independent office
key: commission independent office



### NLU

Now to work on misspellings and synonyms.
We'll use fuzzywuzzy over SpellChecker since fuzzywuzzy calculates the similarity scores between strings, This will help match user queries even if they contain typos

In [29]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [30]:
from spellchecker import SpellChecker

### Debugging Code

In [31]:
combined_sections = {**chapter_13_sections, **chapter_14_sections, **chapter_15_sections}

In [32]:
combined_sections

{'principle': 'Values and principles of public service.\n232. (1) The values and principles of public service include—\n(a) high standards of professional ethics;\n(b) efficient, effective and economic use of resources;\n(c) responsive, prompt, effective, impartial and equitable\nprovision of services;\n(d) involvement of the people in the process of policy making;\n(e) accountability for administrative acts;\n(f) transparency and provision to the public of timely, accurate\ninformation;\n(g) subject to paragraphs (h) and (i), fair competition and merit as\nthe basis of appointments and promotions;\n(h) representation of Kenya’s diverse communities; and\n(i) affording adequate and equal opportunities for appointment,\ntraining and advancement, at all levels of the public service,\nof—\n(i) men and women;\n(ii) the members of all ethnic groups; and\n(iii) persons with disabilities.\n(2) The values and principles of public service apply to public\nservice in—\n(a) all State organs in bot

In [33]:
# Define a dictionary of synonyms for keywords
synonyms = {
    "principle": ["principle", "values and principles of public service", "public service values", "core principles of public service"],
    "public service commission": ["public service commission", "the public service commission", "public service authority", "government service commission"],
    "functions": ["functions", "functions and powers of the public service commission", "roles of the public service commission", "duties of the public service commission"],
    "staffing": ["staffing", "staffing of county governments", "county government staffing", "human resources for county governments"],
    "protection": ["protection", "protection of public officers", "safeguarding public officers", "security for public officials"],
    "teachers service commission": ["teachers service commission", "commission for teachers", "educators service commission"],
    "national security": ["national security", "principles of national security"],
    "national security organs": ["national security organs", "security organs", "national security agencies"],
    "national security council": ["national security council", "establishment of the national security council", "national security council formation", "creation of the national security council"],
    "defence forces": ["defence forces and defence council", "defence forces establishment", "formation of defence forces and council"],
    "national intelligence service": ["national intelligence service", "establishment of national intelligence service", "national intelligence service creation", "formation of national intelligence service"],
    "national police service": ["national police service", "establishment of the national police service", "national police service formation", "creation of the national police service"],
    "functions national police service": ["functions national police service", "objects and functions of the national police service", "roles of the national police service", "duties and functions of the national police"],
    "command national police service": ["command national police service", "command of the national police service", "national police service command", "leadership of the national police service"],
    "national police service commission": ["national police service commission", "police service commission", "commission for the national police service"],
    "other police services": ["other police services", "additional police services", "alternative police services"],
    "application of chapter": ["application of chapter", "chapter application", "provisions of the chapter"],
    "commission independent office": ["commission independent office", "commissions and independent offices", "independent commissions", "commission offices"],
    "term of office": ["term of office", "terms of office", "office terms", "tenure conditions"],
    "removal from office": ["removal from office", "dismissal from office", "termination of office"],
    "general function power": ["general function power", "overall powers", "general powers and functions"],
    "incorporation commission independent office": ["incorporation commission independent office", "incorporation commissions and independent offices", "establishment of commissions", "setting up independent offices"],
    "reporting commission independent office": ["reporting commissions independent office", "reporting commissions and independent offices", "commissions reporting", "reporting by independent offices"]
}



qa_mapping = {
    "principle": "principle",
    "public service commission": "public service commission",
    "functions": "functions",
    "staffing": "staffing",
    "protection": "protection",
    "teachers service commission": "teachers service commission",
    "national security": "national security",
    "national security organs": "national security organs",
    "national security council": "national security council",
    "defence forces": "defence forces",
    "national intelligence service": "national intelligence service",
    "national police service": "national police service",
    "functions national police service": "functions national police service",
    "command national police service": "command national police service",
    "national police service commission": "national police service commission",
    "other police services": "other police services",
    "application of chapter": "application of chapter",
    "commission independent office": "commission independent office",
    "term of office": "term of office",
    "removal from office": "removal from office",
    "general function power": "general function power",
    "incorporation commission independent office": "incorporation commission independent office",
    "reporting commission independent office": "reporting commission independent office"
}


# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)


    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word) # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)

    return corrected_input



# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line


    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Only return section value of the key

    print("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)

    if section_key:
        # Print the section key being looked up
        print(f"Section key found: {section_key}")  # Debugging line
        # Retrieve the relevant section from the specified chapter
        section_value = combined_sections.get(section_key)
        print(f"Retrieved section value: {section_value}")  # Debugging line
        return section_value if section_value else "Section not found."




# Example
user_query = "What about the term of office?"
answer = answer_question_nlp(user_query, combined_sections, qa_mapping, synonyms)
answer


Processed Query: term office
Corrected Query: term office
Checking key: principle, value: principle
Trying synonym: principle
Trying synonym: values and principles of public service
Trying synonym: public service values
Trying synonym: core principles of public service
Checking key: public service commission, value: public service commission
Trying synonym: public service commission
Trying synonym: the public service commission
Trying synonym: public service authority
Trying synonym: government service commission
Checking key: functions, value: functions
Trying synonym: functions
Trying synonym: functions and powers of the public service commission
Trying synonym: roles of the public service commission
Trying synonym: duties of the public service commission
Checking key: staffing, value: staffing
Trying synonym: staffing
Trying synonym: staffing of county governments
Trying synonym: county government staffing
Trying synonym: human resources for county governments
Checking key: protecti

In [34]:
# Combine sections from different chapters into a single dictionary
combined_sections = {**chapter_13_sections, **chapter_14_sections, **chapter_15_sections}

# Define a dictionary of synonyms for keywords
synonyms = {
    "principle": ["principle", "values and principles of public service", "public service values", "core principles of public service"],
    "public service commission": ["public service commission", "the public service commission", "public service authority", "government service commission"],
    "functions": ["functions", "functions and powers of the public service commission", "roles of the public service commission", "duties of the public service commission"],
    "staffing": ["staffing", "staffing of county governments", "county government staffing", "human resources for county governments"],
    "protection": ["protection", "protection of public officers", "safeguarding public officers", "security for public officials"],
    "teachers service commission": ["teachers service commission", "commission for teachers", "educators service commission"],
    "national security": ["national security", "principles of national security"],
    "national security organs": ["national security organs", "security organs", "national security agencies"],
    "national security council": ["national security council", "establishment of the national security council", "national security council formation", "creation of the national security council"],
    "defence forces": ["defence forces and defence council", "defence forces establishment", "formation of defence forces and council"],
    "national intelligence service": ["national intelligence service", "establishment of national intelligence service", "national intelligence service creation", "formation of national intelligence service"],
    "national police service": ["national police service", "establishment of the national police service", "national police service formation", "creation of the national police service"],
    "functions national police service": ["functions national police service", "objects and functions of the national police service", "roles of the national police service", "duties and functions of the national police"],
    "command national police service": ["command national police service", "command of the national police service", "national police service command", "leadership of the national police service"],
    "national police service commission": ["national police service commission", "police service commission", "commission for the national police service"],
    "other police services": ["other police services", "additional police services", "alternative police services"],
    "application of chapter": ["application of chapter", "chapter application", "provisions of the chapter"],
    "commission independent office": ["commission independent office", "commissions and independent offices", "independent commissions", "commission offices"],
    "term of office": ["term of office", "terms of office", "office terms", "tenure conditions"],
    "removal from office": ["removal from office", "dismissal from office", "termination of office"],
    "general function power": ["general function power", "overall powers", "general powers and functions"],
    "incorporation commission independent office": ["incorporation commission independent office", "incorporation commissions and independent offices", "establishment of commissions", "setting up independent offices"],
    "reporting commission independent office": ["reporting commissions independent office", "reporting commissions and independent offices", "commissions reporting", "reporting by independent offices"]
}


qa_mapping = {
    "principle": "principle",
    "public service commission": "public service commission",
    "functions": "functions",
    "staffing": "staffing",
    "protection": "protection",
    "teachers service commission": "teachers service commission",
    "national security": "national security",
    "national security organs": "national security organs",
    "national security council": "national security council",
    "defence forces": "defence forces",
    "national intelligence service": "national intelligence service",
    "national police service": "national police service",
    "functions national police service": "functions national police service",
    "command national police service": "command national police service",
    "national police service commission": "national police service commission",
    "other police services": "other police services",
    "application of chapter": "application of chapter",
    "commission independent office": "commission independent office",
    "term of office": "term of office",
    "removal from office": "removal from office",
    "general function power": "general function power",
    "incorporation commission independent office": "incorporation commission independent office",
    "reporting commission independent office": "reporting commission independent office"
}



# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)

    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word)  # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)
    return corrected_input

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line

    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms.get(key, [key]):
           print(f"Trying synonym: {synonym}")  # Debugging line
           if synonym in corrected_query:
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key
        else:
            print(f"No match for synonym: {synonym} in {corrected_query}")  # Debugging line

# Answer function with synonym matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)

    if section_key:
        # Check what section key was retrieved
        print(f"Retrieved section key: {section_key}")  # Debugging line
        # Retrieve the relevant section from the combined sections
        section_title = qa_mapping.get(section_key)
        print(f"Section title being looked up: {section_title}")  # Debugging line

        if section_title:
            return sections.get(section_title, "Section not found.")

    print("No section key found")  # Debugging line if no key is found
    return "Section not found."


# Example
user_query = "What about the national police service?"
answer = answer_question_nlp(user_query, combined_sections, qa_mapping, synonyms)
print(answer)


Processed Query: national police service
Corrected Query: national police service
Checking key: principle, value: principle
Trying synonym: principle
Trying synonym: values and principles of public service
Trying synonym: public service values
Trying synonym: core principles of public service
No match for synonym: core principles of public service in national police service
Checking key: public service commission, value: public service commission
Trying synonym: public service commission
Trying synonym: the public service commission
Trying synonym: public service authority
Trying synonym: government service commission
No match for synonym: government service commission in national police service
Checking key: functions, value: functions
Trying synonym: functions
Trying synonym: functions and powers of the public service commission
Trying synonym: roles of the public service commission
Trying synonym: duties of the public service commission
No match for synonym: duties of the public ser

In [35]:
# Example 2
user_query = "What about the terms of office?"
answer = answer_question_nlp(user_query, combined_sections, qa_mapping, synonyms)
print(answer)


Processed Query: term office
Corrected Query: term office
Checking key: principle, value: principle
Trying synonym: principle
Trying synonym: values and principles of public service
Trying synonym: public service values
Trying synonym: core principles of public service
No match for synonym: core principles of public service in term office
Checking key: public service commission, value: public service commission
Trying synonym: public service commission
Trying synonym: the public service commission
Trying synonym: public service authority
Trying synonym: government service commission
No match for synonym: government service commission in term office
Checking key: functions, value: functions
Trying synonym: functions
Trying synonym: functions and powers of the public service commission
Trying synonym: roles of the public service commission
Trying synonym: duties of the public service commission
No match for synonym: duties of the public service commission in term office
Checking key: sta

In [36]:
# Combine sections from different chapters into a single dictionary
combined_sections = {**chapter_13_sections, **chapter_14_sections, **chapter_15_sections}

# Define a dictionary of synonyms for keywords
synonyms = {
    "principle": ["principle", "values and principles of public service", "public service values", "core principles of public service"],
    "public service commission": ["public service commission", "the public service commission", "public service authority", "government service commission"],
    "functions": ["functions", "functions and powers of the public service commission", "roles of the public service commission", "duties of the public service commission"],
    "staffing": ["staffing", "staffing of county governments", "county government staffing", "human resources for county governments"],
    "protection": ["protection", "protection of public officers", "safeguarding public officers", "security for public officials"],
    "teachers service commission": ["teachers service commission", "commission for teachers", "educators service commission"],
    "national security": ["national security", "principles of national security"],
    "national security organs": ["national security organs", "security organs", "national security agencies"],
    "national security council": ["national security council", "establishment of the national security council", "national security council formation", "creation of the national security council"],
    "defence forces": ["defence forces and defence council", "defence forces establishment", "formation of defence forces and council"],
    "national intelligence service": ["national intelligence service", "establishment of national intelligence service", "national intelligence service creation", "formation of national intelligence service"],
    "national police service": ["national police service", "establishment of the national police service", "national police service formation", "creation of the national police service"],
    "functions national police service": ["functions national police service", "objects and functions of the national police service", "roles of the national police service", "duties and functions of the national police"],
    "command national police service": ["command national police service", "command of the national police service", "national police service command", "leadership of the national police service"],
    "national police service commission": ["national police service commission", "police service commission", "commission for the national police service"],
    "other police services": ["other police services", "additional police services", "alternative police services"],
    "application of chapter": ["application of chapter", "chapter application", "provisions of the chapter"],
    "commission independent office": ["commission independent office", "commissions and independent offices", "independent commissions", "commission offices"],
    "term of office": ["term of office", "terms of office", "office terms", "tenure conditions"],
    "removal from office": ["removal from office", "dismissal from office", "termination of office"],
    "general function power": ["general function power", "overall powers", "general powers and functions"],
    "incorporation commission independent office": ["incorporation commission independent office", "incorporation commissions and independent offices", "establishment of commissions", "setting up independent offices"],
    "reporting commission independent office": ["reporting commissions independent office", "reporting commissions and independent offices", "commissions reporting", "reporting by independent offices"]
}


qa_mapping = {
    "principle": "principle",
    "public service commission": "public service commission",
    "functions": "functions",
    "staffing": "staffing",
    "protection": "protection",
    "teachers service commission": "teachers service commission",
    "national security": "national security",
    "national security organs": "national security organs",
    "national security council": "national security council",
    "defence forces": "defence forces",
    "national intelligence service": "national intelligence service",
    "national police service": "national police service",
    "functions national police service": "functions national police service",
    "command national police service": "command national police service",
    "national police service commission": "national police service commission",
    "other police services": "other police services",
    "application of chapter": "application of chapter",
    "commission independent office": "commission independent office",
    "term of office": "term of office",
    "removal from office": "removal from office",
    "general function power": "general function power",
    "incorporation commission independent office": "incorporation commission independent office",
    "reporting commission independent office": "reporting commission independent office"
}

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)

    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word)  # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)
    return corrected_input

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line

    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:  # Convert synonym to lower case for matching
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Return the key if a match is found

    print("No match found")  # Debugging line if no match is found
    return None  # Return None if no matches were found

# Answer function with synonym matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)

    if section_key:
        # Print the section key being looked up
        print(f"Section key found: {section_key}")  # Debugging line
        # Retrieve the relevant section from the specified chapter
        section_value = sections.get(section_key)  # Use sections instead of combined_sections if that's the variable name
        print(f"Retrieved section value: {section_value}")  # Debugging line
        return section_value if section_value else "Section not found."

    return "Sorry, I couldn't find an answer to your question."

# Example
user_query = "What about the staffing?"
answer = answer_question_nlp(user_query, combined_sections, qa_mapping, synonyms)
print(answer)  # Display the answer


Processed Query: staffing
Corrected Query: staffing
Checking key: principle, value: principle
Trying synonym: principle
Trying synonym: values and principles of public service
Trying synonym: public service values
Trying synonym: core principles of public service
Checking key: public service commission, value: public service commission
Trying synonym: public service commission
Trying synonym: the public service commission
Trying synonym: public service authority
Trying synonym: government service commission
Checking key: functions, value: functions
Trying synonym: functions
Trying synonym: functions and powers of the public service commission
Trying synonym: roles of the public service commission
Trying synonym: duties of the public service commission
Checking key: staffing, value: staffing
Trying synonym: staffing
Match found with synonym: staffing
Section key found: staffing
Retrieved section value: Staffing of county governments.
235. (1) A county government is responsible, within a

## Precise code

In [37]:
chapter_14_sections

{'national security': 'Principles of national security.\n238. (1) National security is the protection against internal and\nexternal threats to Kenya’s territorial integrity and sovereignty, its\npeople, their rights, freedoms, property, peace, stability and prosperity,\nand other national interests.\n(2) The national security of Kenya shall be promoted and\nguaranteed in accordance with the following principles—\n(a) national security is subject to the authority of this Constitution\nand Parliament;\n(b) national security shall be pursued in compliance with the law\nand with the utmost respect for the rule of law, democracy,\nhuman rights and fundamental freedoms;\n(c) in performing their functions and exercising their powers,\nnational security organs shall respect the diverse culture of\nthe communities within Kenya; and\n(d) recruitment by the national security organs shall reflect the\ndiversity of the Kenyan people in equitable proportions.',
 'national security organs': 'Nationa

In [38]:
# Combine sections from different chapters into a single dictionary
combined_sections = {**chapter_13_sections, **chapter_14_sections, **chapter_15_sections}

# Define a dictionary of synonyms for keywords
synonyms = {
    "principle": ["principle", "values and principles of public service", "public service values", "core principles of public service"],
    "public service commission": ["public service commission", "the public service commission", "public service authority", "government service commission"],
    "functions": ["functions", "functions and powers of the public service commission", "roles of the public service commission", "duties of the public service commission"],
    "staffing": ["staffing", "staffing of county governments", "county government staffing", "human resources for county governments"],
    "protection": ["protection", "protection of public officers", "safeguarding public officers", "security for public officials"],
    "teachers service commission": ["teachers service commission", "commission for teachers", "educators service commission"],
    "national security": ["national security", "principles of national security"],
    "national security organs": ["national security organs", "security organs", "national security agencies"],
    "national security council": ["national security council", "establishment of the national security council", "national security council formation", "creation of the national security council"],
    "defence forces": ["defence forces and defence council", "defence forces establishment", "formation of defence forces and council"],
    "national intelligence service": ["national intelligence service", "establishment of national intelligence service", "national intelligence service creation", "formation of national intelligence service"],
    "national police service": ["national police service", "establishment of the national police service", "national police service formation", "creation of the national police service"],
    "functions national police service": ["functions national police service", "objects and functions of the national police service", "roles of the national police service", "duties and functions of the national police"],
    "command national police service": ["command national police service", "command of the national police service", "national police service command", "leadership of the national police service"],
    "national police service commission": ["national police service commission", "police service commission", "commission for the national police service"],
    "other police services": ["other police services", "additional police services", "alternative police services"],
    "application of chapter": ["application of chapter", "chapter application", "provisions of the chapter"],
    "commission independent office": ["commission independent office", "commissions and independent offices", "independent commissions", "commission offices"],
    "term of office": ["term of office", "terms of office", "office terms", "tenure conditions"],
    "removal from office": ["removal from office", "dismissal from office", "termination of office"],
    "general function power": ["general function power", "overall powers", "general powers and functions"],
    "incorporation commission independent office": ["incorporation commission independent office", "incorporation commissions and independent offices", "establishment of commissions", "setting up independent offices"],
    "reporting commission independent office": ["reporting commissions independent office", "reporting commissions and independent offices", "commissions reporting", "reporting by independent offices"]
}


qa_mapping = {
    "principle": "principle",
    "public service commission": "public service commission",
    "functions": "functions",
    "staffing": "staffing",
    "protection": "protection",
    "teachers service commission": "teachers service commission",
    "national security": "national security",
    "national security organs": "national security organs",
    "national security council": "national security council",
    "defence forces": "defence forces",
    "national intelligence service": "national intelligence service",
    "national police service": "national police service",
    "functions national police service": "functions national police service",
    "command national police service": "command national police service",
    "national police service commission": "national police service commission",
    "other police services": "other police services",
    "application of chapter": "application of chapter",
    "commission independent office": "commission independent office",
    "term of office": "term of office",
    "removal from office": "removal from office",
    "general function power": "general function power",
    "incorporation commission independent office": "incorporation commission independent office",
    "reporting commission independent office": "reporting commission independent office"
}

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)

    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word)  # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)
    return corrected_input

# Match with synonym support
def match_with_synonyms(query, qa_mapping, synonyms):
    processed_query = preprocess_query(query)
    print(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    print(f"Corrected Query: {corrected_query}")  # Debugging line

    for key, value in qa_mapping.items():
        print(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms.get(key, [key]):
            print(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:  # Convert synonym to lower case for matching
                print(f"Match found with synonym: {synonym}")  # Debugging line
                return key  # Return the key if a match is found

    print("No match found")  # Debugging line if no match is found
    return None  # Return None if no matches were found

# Answer function with synonym matching
def answer_question_nlp(query, combined_sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)

    if section_key:
        # Print the section key being looked up
        print(f"Section key found: {section_key}")  # Debugging line
        # Retrieve the relevant section from the specified chapter
        section_value = combined_sections.get(section_key)  # Use sections instead of combined_sections if that's the variable name
        print(f"Retrieved section value: {section_value}")  # Debugging line
        return section_value if section_value else "Section not found."

    return "Sorry, I couldn't find an answer to your question."

# Example
user_query = "What about the terms of office"
answer = answer_question_nlp(user_query, combined_sections, qa_mapping, synonyms)
print(answer)  # Display the answer


Processed Query: term office
Corrected Query: term office
Checking key: principle, value: principle
Trying synonym: principle
Trying synonym: values and principles of public service
Trying synonym: public service values
Trying synonym: core principles of public service
Checking key: public service commission, value: public service commission
Trying synonym: public service commission
Trying synonym: the public service commission
Trying synonym: public service authority
Trying synonym: government service commission
Checking key: functions, value: functions
Trying synonym: functions
Trying synonym: functions and powers of the public service commission
Trying synonym: roles of the public service commission
Trying synonym: duties of the public service commission
Checking key: staffing, value: staffing
Trying synonym: staffing
Trying synonym: staffing of county governments
Trying synonym: county government staffing
Trying synonym: human resources for county governments
Checking key: protecti

In [39]:
list(combined_sections.keys())[10:]

['national intelligence service',
 'national police service',
 'functions national police service',
 'command national police service',
 'national police service commission',
 'other police services',
 'application of chapter',
 'commission independent office',
 'term of office',
 'removal from office',
 'general function power',
 'incorporation commission independent office',
 'reporting commission independent office']

##Test for the synonyms


In [40]:
user_query = "What are the defence forces?"
answer = answer_question_nlp(user_query,combined_sections,qa_mapping,synonyms)
print(answer)

Processed Query: defence force
Corrected Query: defense force
Checking key: principle, value: principle
Trying synonym: principle
Trying synonym: values and principles of public service
Trying synonym: public service values
Trying synonym: core principles of public service
Checking key: public service commission, value: public service commission
Trying synonym: public service commission
Trying synonym: the public service commission
Trying synonym: public service authority
Trying synonym: government service commission
Checking key: functions, value: functions
Trying synonym: functions
Trying synonym: functions and powers of the public service commission
Trying synonym: roles of the public service commission
Trying synonym: duties of the public service commission
Checking key: staffing, value: staffing
Trying synonym: staffing
Trying synonym: staffing of county governments
Trying synonym: county government staffing
Trying synonym: human resources for county governments
Checking key: prot

## Testing for misspellings

In [41]:
user_query = "What are the values of the Pubic Service Commission?"
answer = answer_question_nlp(user_query,sections,qa_mapping,synonyms)
print(answer)

Processed Query: value pubic service commission
Corrected Query: value pubic service commission
Checking key: principle, value: principle
Trying synonym: principle
Trying synonym: values and principles of public service
Trying synonym: public service values
Trying synonym: core principles of public service
Checking key: public service commission, value: public service commission
Trying synonym: public service commission
Trying synonym: the public service commission
Trying synonym: public service authority
Trying synonym: government service commission
Checking key: functions, value: functions
Trying synonym: functions
Trying synonym: functions and powers of the public service commission
Trying synonym: roles of the public service commission
Trying synonym: duties of the public service commission
Checking key: staffing, value: staffing
Trying synonym: staffing
Trying synonym: staffing of county governments
Trying synonym: county government staffing
Trying synonym: human resources for cou

The code efficiently corrects the misspelt words and also considers the synonyms used.

### Combining all chapter sections into a singlr dictionary

In [42]:
combined_sections = {
    **chapter_13_sections,
    **chapter_14_sections,
    **chapter_15_sections

}

combined_sections.keys()

dict_keys(['principle', 'public service commission', 'functions', 'staffing', 'protection', 'teacher service commission', 'national security', 'national security organs', 'national security council', 'defence forces', 'national intelligence service', 'national police service', 'functions national police service', 'command national police service', 'national police service commission', 'other police services', 'application of chapter', 'commission independent office', 'term of office', 'removal from office', 'general function power', 'incorporation commission independent office', 'reporting commission independent office'])

In [44]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Log message format
)

## Define synonym mapping
synonyms = {
    "principle": ["principle", "values and principles of public service", "public service values", "core principles of public service"],
    "public service commission": ["public service commission", "the public service commission", "public service authority", "government service commission"],
    "functions": ["functions", "functions and powers of the public service commission", "roles of the public service commission", "duties of the public service commission"],
    "staffing": ["staffing", "staffing of county governments", "county government staffing", "human resources for county governments"],
    "protection": ["protection", "protection of public officers", "safeguarding public officers", "security for public officials"],
    "teachers service commission": ["teachers service commission", "commission for teachers", "educators service commission"],
    "national security": ["national security", "principles of national security"],
    "national security organs": ["national security organs", "security organs", "national security agencies"],
    "national security council": ["national security council", "establishment of the national security council", "national security council formation", "creation of the national security council"],
    "defence forces": ["defence forces and defence council", "defence forces establishment", "formation of defence forces and council"],
    "national intelligence service": ["national intelligence service", "establishment of national intelligence service", "national intelligence service creation", "formation of national intelligence service"],
    "national police service": ["national police service", "establishment of the national police service", "national police service formation", "creation of the national police service"],
    "functions national police service": ["functions national police service", "objects and functions of the national police service", "roles of the national police service", "duties and functions of the national police"],
    "command national police service": ["command national police service", "command of the national police service", "national police service command", "leadership of the national police service"],
    "national police service commission": ["national police service commission", "police service commission", "commission for the national police service"],
    "other police services": ["other police services", "additional police services", "alternative police services"],
    "application of chapter": ["application of chapter", "chapter application", "provisions of the chapter"],
    "commission independent office": ["commission independent office", "commissions and independent offices", "independent commissions", "commission offices"],
    "term of office": ["term of office", "terms of office", "office terms", "tenure conditions"],
    "removal from office": ["removal from office", "dismissal from office", "termination of office"],
    "general function power": ["general function power", "overall powers", "general powers and functions"],
    "incorporation commission independent office": ["incorporation commission independent office", "incorporation commissions and independent offices", "establishment of commissions", "setting up independent offices"],
    "reporting commission independent office": ["reporting commissions independent office", "reporting commissions and independent offices", "commissions reporting", "reporting by independent offices"]
}

# QA mapping
qa_mapping = {
    "principle": "principle",
    "public service commission": "public service commission",
    "functions": "functions",
    "staffing": "staffing",
    "protection": "protection",
    "teachers service commission": "teachers service commission",
    "national security": "national security",
    "national security organs": "national security organs",
    "national security council": "national security council",
    "defence forces": "defence forces",
    "national intelligence service": "national intelligence service",
    "national police service": "national police service",
    "functions national police service": "functions national police service",
    "command national police service": "command national police service",
    "national police service commission": "national police service commission",
    "other police services": "other police services",
    "application of chapter": "application of chapter",
    "commission independent office": "commission independent office",
    "term of office": "term of office",
    "removal from office": "removal from office",
    "general function power": "general function power",
    "incorporation commission independent office": "incorporation commission independent office",
    "reporting commission independent office": "reporting commission independent office"
}


sections = combined_sections

# Preprocess the query
def preprocess_query(query):
    doc = nlp(query)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

# Function to correct the spelling
def correct_spelling(processed_query):
    spell = SpellChecker()
    words = processed_query.split()
    # Find misspelled words
    misspelled_words = spell.unknown(words)

    corrected_words = []
    for word in words:
        # Correct the word if it's misspelled
        if word in misspelled_words:
            corrected_word = spell.correction(word)  # Get the most likely correction
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)

    # Reconstruct the initial sentence
    corrected_input = " ".join(corrected_words)
    return corrected_input

# Match the words with synonym support
def match_with_synonyms(query, qa_mapping, synonyms, threshold=70):
    processed_query = preprocess_query(query)
    logging.debug(f"Processed Query: {processed_query}")  # Debugging line

    # Correct spelling in the processed query
    corrected_query = correct_spelling(processed_query)
    logging.debug(f"Corrected Query: {corrected_query}")  # Debugging line

    for key, value in qa_mapping.items():
        logging.debug(f"Checking key: {key}, value: {value}")  # Debugging line
        for synonym in synonyms.get(key, [key]):
            logging.debug(f"Trying synonym: {synonym}")  # Debugging line
            if synonym in corrected_query:
                logging.debug(f"Match found with synonym: {synonym}")  # Debugging line
                return value  # Only return section value of the key

    logging.debug("No match found")  # Debugging line if no match is found
    return None

# Answer function with synonym and fuzzy matching
def answer_question_nlp(query, sections, qa_mapping, synonyms):
    section_key = match_with_synonyms(query, qa_mapping, synonyms)

    if section_key:
        # Retrieve the relevant section from the specified chapter
        return sections.get(section_key, "Section not found.")

    logging.error("No answer found for the question.")  # Log an error if no answer is found
    return "Sorry, I couldn't find an answer to your question."

# Sample usage
if __name__ == "__main__":
    user_query = "What does it say about principles?"
    answer = answer_question_nlp(user_query, sections, qa_mapping, synonyms)
    print(answer)


2024-11-02 20:43:44,084 - DEBUG - Processed Query: principle


2024-11-02 20:43:44,428 - DEBUG - Corrected Query: principle
2024-11-02 20:43:44,428 - DEBUG - Checking key: principle, value: principle
2024-11-02 20:43:44,428 - DEBUG - Trying synonym: principle
2024-11-02 20:43:44,428 - DEBUG - Match found with synonym: principle


Values and principles of public service.
232. (1) The values and principles of public service include—
(a) high standards of professional ethics;
(b) efficient, effective and economic use of resources;
(c) responsive, prompt, effective, impartial and equitable
provision of services;
(d) involvement of the people in the process of policy making;
(e) accountability for administrative acts;
(f) transparency and provision to the public of timely, accurate
information;
(g) subject to paragraphs (h) and (i), fair competition and merit as
the basis of appointments and promotions;
(h) representation of Kenya’s diverse communities; and
(i) affording adequate and equal opportunities for appointment,
training and advancement, at all levels of the public service,
of—
(i) men and women;
(ii) the members of all ethnic groups; and
(iii) persons with disabilities.
(2) The values and principles of public service apply to public
service in—
(a) all State organs in both levels of government; and
(b) all 